In [3]:
# -*- coding: utf-8 -*-
# python 3.6
# 测试系统，Win10
# Author:walter
# 实现自动下载影视剧
# V1.0 当前只针对www.loldytt.com
# 如果您有经验优化，改进此脚本，请不吝指教
# mail: arcayi@qq.com

from selenium import webdriver
import requests
from lxml import etree
import re
import os
from win32com.client import Dispatch
from urllib import parse

from  multiprocessing import Pool#, Process, Manager

search_url = {
    'loldytt': "https://www.loldytt.com/search/index.asp?keyword=%s"
}
search_xpath = {
    'loldytt': '/html/body/center/div[6]/ol/label/a'
#               /html/body/center/div[6]/ol[1]/label/a
#               /html/body/center/div[6]/ol[3]/label/a
}

download_xpath = {
#     'loldytt' : '/html/body/center/div#liebiao/div/div.con4'
    'loldytt' : '//ul[@class="downurl"]/li/a'
#     'loldytt' : '//ul[@class="downurl"]'
}

def search_video(search_keywords, encoding='gbk', source_site = "loldytt"):
    video_name_urlEncode = parse.quote(search_keywords,encoding=encoding)
    sUrl = search_url[source_site]%(video_name_urlEncode)
#     print('search url:\n\t', sUrl, '\n')
    html = requests.get(sUrl).content
    selector = etree.HTML(html)
    total_videos = selector.xpath(search_xpath[source_site])
    download_pages = {}
    for iVideo in total_videos:
        download_pages[iVideo.text] = iVideo.xpath('@href')[0]
#     print(download_pages)
    return(download_pages)

def get_download_url_list(download_url, source_site = "loldytt"):
    html = requests.get(download_url).content
#     print(html)
    selector = etree.HTML(html)
    total_video_downloads = selector.xpath(download_xpath[source_site])
#     print( '链接数：', len(total_video_downloads), '\n' )
    download_urls = {}
    for iVideo in total_video_downloads:
#         print(etree.tostring(iVideo, pretty_print=False),'\n')
        download_urls[iVideo.text] = iVideo.xpath('@href')[0]
    return(download_urls)

def downloadUrlEncode(url, target='thunder'):
#     print(url)
    if(target == 'thunder'):
        result = "thunder://"+(base64.b64encode(('AA'+url+'ZZ').encode('gb2312'))).decode()
    elif(target == 'flashget'):
        flashget="[FLASHGET]"+url+"[FLASHGET]"
        result="flashget://"+(base64.b64encode(flashget.encode('gb2312'))).decode()+"&abc"
    elif(target == 'qqdl'):
        result="qqdl://"+(base64.b64encode(url.encode('gb2312'))).decode()
    return(result)
    
def addTasktoXunlei(down_url,course_infos):
#     flag = False
#     o = Dispatch("ThunderAgent.Agent.1")
#     if down_url:
#         course_path = os.getcwd()
#         try:
#             #AddTask("下载地址", "另存文件名", "保存目录","任务注释","引用地址","开始模式", "只从原始地址下载","从原始地址下载线程数")
#             o.AddTask(down_url, '', course_path, "", "", -1, 0, 5)
#             o.CommitTasks()
#             flag = True
#         except Exception:
#             print(Exception.message)
#             print(" AddTask is fail!")
#     return flag

    os.chdir("C:\\Program Files (x86)\\Thunder Network\\Thunder\\Program\\")
    os.system("Thunder.exe \"%s\""%down_url)


def downloadFromSearch( keyword ):
    download_pages = search_video( keyword )
    for vName in download_pages:
        print(vName, download_pages[vName])
        downloadFromInfoUrl( download_pages[vName] )

def downloadFromInfoUrl(infoUrl):
        download_urls = get_download_url_list( infoUrl )
#         print(download_urls,'\n')
        for i in download_urls:
#             print( i, '\t', download_urls[i],'\n')            
            addTasktoXunlei( download_urls[i], course_infos=None)

### 搜索影视剧集名称，并下载所有资源

In [2]:
video_name = input('Please input the video name:\n')
print ('begin to search...')
print ('---------------------------------')
downloadFromSearch(video_name)

Please input the video name:

begin to search...
---------------------------------


### 根据影视剧集的介绍页面，下载所有资源

In [3]:
url = input('Please input the video info page url:\n')
print ('begin to download...')
print ('---------------------------------')
downloadFromInfoUrl( url )

Please input the video info page url:
https://www.loldytt.com/Dianshiju/DJSSMZHXLYJSLM2/
begin to download...
---------------------------------
第01-06集 	 magnet:?xt=urn:btih:DF5CB60D79FE5764ACCB0AEBA8105222F622CBF1 

第01集 	 thunder://QUFodHRwOi8vZGw5NC44MHMuY29tLmNvOjk5OS8xNzEyL1u08778yqZTTVnWrkjQpUzS91212jAxvK8vW7TzvvzKplNNWdauSNClTNL3XbXaMDG8r19iZC5tcDRaWg== 

第02集 	 thunder://QUFodHRwOi8vZGw5NC44MHMuY29tLmNvOjk5OS8xNzEyL1u08778yqZTTVnWrkjQpUzS91212jAyvK8vW7TzvvzKplNNWdauSNClTNL3XbXaMDK8r19iZC5tcDRaWg== 

第03集 	 thunder://QUFodHRwOi8vZGw5NC44MHMuY29tLmNvOjk5OS8xNzEyL1u08778yqZTTVnWrkjQpUzS91212jAzvK8vW7TzvvzKplNNWdauSNClTNL3XbXaMDO8r19iZC5tcDRaWg== 

第04集 	 thunder://QUFodHRwOi8vZGw5NC44MHMuY29tLmNvOjk5OS8xNzEyL1u08778yqZTTVnWrkjQpUzS91212jA0vK8vW7TzvvzKplNNWdauSNClTNL3XbXaMDS8r19iZC5tcDRaWg== 

第05集 	 thunder://QUFodHRwOi8vZGw5NC44MHMuY29tLmNvOjk5OS8xNzEyL1u08778yqZTTVnWrkjQpUzS91212jA1vK8vW7TzvvzKplNNWdauSNClTNL3XbXaMDW8r19iZC5tcDRaWg== 

第06集 	 thunder://QUFodHRwOi8vZGw5NC44MHMuY2

### 根据影视剧集的介绍页面列表，下载所有资源

In [5]:
urls = [
    'https://www.loldytt.com/Dianshiju/DJSSMZHXLYJSLM2/', #大军师司马懿之虎啸龙吟/军师联盟2
    'http://www.loldytt.com/Zuixinmeiju/WJCQDWJ/', #维京传奇第五季
    'http://www.loldytt.com/Zuixinmeiju/LBQDMSEFRDYJ/', # 了不起的麦瑟尔夫人第一季
    'https://www.loldytt.com/Zuixinmeiju/SHDBZDSJ/', # 生活大爆炸第十季
    'https://www.loldytt.com/Zuixinmeiju/SHDBZDSYJ/', # 生活大爆炸第十一季
    'http://www.loldytt.com/Zuixinmeiju/JQSNFLJDYJ/', # 机器少女法兰姬第一季
    'https://www.loldytt.com/Zuixinmeiju/SNXEDDYJ/', #少年谢尔顿第一季
    'https://www.loldytt.com/Zuixinmeiju/BEDKBDKDSJ/', # 波尔达克/波达克第三季
]
print ('开始处理...')
print ('---------------------------------')
processNumber = 5
pool = Pool( processNumber )  #允许进程池同时放入5个进程
#     p.map(downloadFromInfoUrl, urls )
for url in urls:
    print(url)
    downloadFromInfoUrl(url) 
#     pool.apply_async(
#         func=downloadFromInfoUrl, args=(url) #, callback = lambda x : dfResult.append(x)
#     )
# pool.close()
# pool.join() #主进程等待所有子进程执行完毕。必须在close()或terminate()之后。
print ('---------------------------------')
print ('准备好下载，请切换到迅雷进行下一步')

开始处理...
---------------------------------
https://www.loldytt.com/Dianshiju/DJSSMZHXLYJSLM2/
http://www.loldytt.com/Zuixinmeiju/WJCQDWJ/
http://www.loldytt.com/Zuixinmeiju/LBQDMSEFRDYJ/
https://www.loldytt.com/Zuixinmeiju/SHDBZDSJ/
https://www.loldytt.com/Zuixinmeiju/SHDBZDSYJ/
http://www.loldytt.com/Zuixinmeiju/JQSNFLJDYJ/
https://www.loldytt.com/Zuixinmeiju/SNXEDDYJ/
https://www.loldytt.com/Zuixinmeiju/BEDKBDKDSJ/
---------------------------------
准备好下载，请切换到迅雷进行下一步
